In [1]:
!pip install mysql-connector-python
import mysql.connector as connector

connection = connector.connect(user="root2", password="yXo_ztj9389HWZr", host="localhost")
cursor = connection.cursor()
cursor.execute("USE littlelemondb")
cursor.execute("SHOW TABLES")
results = cursor.fetchall()
for result in results:
    print(result)

('bookings',)
('customers',)
('menuitems',)
('menus',)
('orderdeliverystatus',)
('orders',)
('ordersview',)
('staff',)


In [35]:
# Task 1: Creating the OrdersView
# In the first task, Little Lemon need you to create a virtual table called OrdersView that focuses on 
# OrderID, Quantity and Cost columns within the Orders table for all orders with a quantity greater than 2. 

viewstmt = """
CREATE VIEW IF NOT EXISTS OrdersView AS
SELECT OrderID, Quantity, TotalCost
FROM Orders
WHERE Quantity > 2;
"""

cursor.execute(viewstmt)

cursor.execute("SELECT * FROM OrdersView")
cols = cursor.column_names
print(cols)
results = cursor.fetchall()
for result in results:
    print(result)

('OrderID', 'Quantity', 'TotalCost')
(3, 3, Decimal('29.97'))
(4, 4, Decimal('39.96'))
(6, 5, Decimal('49.95'))
(8, 3, Decimal('29.97'))
(14, 5, Decimal('49.95'))
(15, 4, Decimal('39.96'))
(17, 3, Decimal('29.97'))
(19, 4, Decimal('39.96'))
(20, 5, Decimal('49.95'))


In [176]:
#Task 2 For your second task, Little Lemon need information from four tables on all customers with orders that cost more than $150.
#Note: my database doesn't have anything over $150, so I multiplied the total cost by 10 to get a result.

sqlstmt = """
SELECT Customers.CustomerID AS CustomerID, Customers.FullName AS FullName,
Orders.OrderID AS OrderID, Orders.TotalCost*10 AS Cost, Menus.Cuisine AS MenuName, MenuItems.Name AS CourseName
FROM Customers
INNER JOIN Bookings ON Bookings.CustomerID = Customers.CustomerID
INNER JOIN Orders ON Orders.OrderID = Bookings.BookingID
INNER JOIN Menus ON Menus.MenuID = Orders.MenuID
INNER JOIN MenuItems ON MenuItems.MenuID = Menus.MenuID
WHERE MenuItems.Type = 'Main Course' AND Orders.TotalCost*10 > 150;

"""

cursor.execute(sqlstmt)
cols = cursor.column_names
print(cols)
results = cursor.fetchall()
for result in results:
    print(result)

('CustomerID', 'FullName', 'OrderID', 'Cost', 'MenuName', 'CourseName')
(1, 'John Doe', 1, Decimal('199.80'), 'Italian', 'Margherita Pizza')
(3, 'Alice Johnson', 3, Decimal('299.70'), 'Chinese', 'Kung Pao Chicken')
(4, 'Bob Brown', 4, Decimal('399.60'), 'Indian', 'Butter Chicken')
(6, 'Diana Evans', 6, Decimal('499.50'), 'Japanese', 'Sushi')
(8, 'Fiona Harris', 8, Decimal('299.70'), 'Greek', 'Moussaka')
(9, 'George King', 9, Decimal('199.80'), 'Spanish', 'Paella')
(12, 'Jack Nelson', 12, Decimal('299.70'), 'Mexican', 'Burrito')
(17, 'Olivia Scott', 17, Decimal('299.70'), 'Thai', 'Green Curry')
(18, 'Paul Turner', 18, Decimal('199.80'), 'Greek', 'Gyro')
(20, 'Rachel Vance', 20, Decimal('499.50'), 'American', 'Hot Dog')


In [177]:
#Task 3 For the third and final task, Little Lemon need you to find all menu items for which more than 2 orders have been placed. You can carry out this task by creating a subquery that lists the menu names from the menus table 
# for any order quantity with more than 2.

sqlstmt = """
SELECT Menus.Cuisine AS MenuName
FROM Menus
WHERE OrderID = ANY
(SELECT OrderID
FROM Orders
WHERE Quantity >2)

"""

cursor.execute(sqlstmt)
cols = cursor.column_names
print(cols)
results = cursor.fetchall()
for result in results:
    print(result)

('MenuName',)
('Chinese',)
('Indian',)
('Japanese',)
('Greek',)
('Mexican',)
('Indian',)
('French',)
('Thai',)
('Spanish',)
('American',)


In [178]:
# Stored Procedures and Prepared Statements
# Task 1: In this first task, Little Lemon need you to create a procedure that displays the maximum ordered quantity in the Orders table. GetMaxQuantity

#Create stored procedure
spstmt = """
CREATE PROCEDURE IF NOT EXISTS GetMaxQuantity()
BEGIN
SELECT Quantity AS 'Max Quantity in Order'
FROM Orders
ORDER BY Quantity DESC
LIMIT 1;
END

"""
cursor.execute(spstmt)

# Call stored prodcedure
sqlstmt = """
CALL GetMaxQuantity

"""

cursor.execute(sqlstmt)
cols = cursor.column_names
print(cols)
results = cursor.fetchall()
for result in results:
    print(result)
cursor.close()


('Max Quantity in Order',)
(5,)


True

In [179]:
#Task 2: In the second task, Little Lemon need you to help them to create a prepared statement called GetOrderDetail. This prepared statement will help to reduce 
#the parsing time of queries. It will also help to secure the database from SQL injections.

connection = connector.connect(user="root2", password="yXo_ztj9389HWZr", host="localhost")
cursor = connection.cursor()
cursor.execute("USE littlelemondb")

#PREPARE statement_Name 'INSERT INTO table1 VALUES (?, ?, ?, ?)';
#EXECUTE statement_name USING @variable;


#Create the Prepared Statement

query = """
PREPARE GetOrderDetails FROM
'SELECT Orders.OrderID AS OrderID, Orders.Quantity AS Quantity, Orders.TotalCost AS Cost
FROM Orders
INNER JOIN Bookings ON Bookings.BookingID = Orders.BookingID
INNER JOIN Customers ON Customers.CustomerID = Bookings.CustomerID
WHERE Customers.CustomerID = ?';
"""

cursor.execute(query)

#run the prepared statement with %id=1

sqlstmt = """
EXECUTE GetOrderDetails USING @id;
"""


cursor.execute("SET @id = 1")
cursor.execute(sqlstmt)
cols = cursor.column_names
print(cols)
results = cursor.fetchall()
for result in results:
    print(result)



('OrderID', 'Quantity', 'Cost')
(1, 2, Decimal('19.98'))


In [3]:
#Task 3 Your third and final task is to create a stored procedure called CancelOrder. Little Lemon want to use this stored procedure to
#delete an order record based on the user input of the order id.


connection = connector.connect(user="root2", password="yXo_ztj9389HWZr", host="localhost")
cursor = connection.cursor()
cursor.execute("USE littlelemondb")

#cursor.execute("DROP PROCEDURE CancelOrder;")

#Create stored procedure
spstmt = """
CREATE PROCEDURE IF NOT EXISTS CancelOrder(IN myorderid INT)
BEGIN

DELETE FROM Orders WHERE OrderID = myorderid ;
SELECT CONCAT('Order ', myorderid, ' is canceled.')  AS message;

END
"""

cursor.execute(spstmt)

# caling the procedure
cursor.callproc('CancelOrder', (16,) )

# Print the output value
for result in cursor.stored_results():
    print("Confirmation")
    print(result.fetchall()[0][0])






Confirmation
Order 16 is canceled.
